In [3]:
!pip install transformers
!pip install -U bitsandbytes
!pip install pandas
!pip install sentencepiece protobuf 
!pip install accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import pandas as pd
import requests
import os
import json
import csv
import torch
import re

In [5]:
from huggingface_hub import login

In [6]:
df_bugs = pd.read_csv("active-bugs.csv")
df_summaries = pd.read_csv("gt-summaries.csv")

In [8]:
print(df_bugs.head())
print(df_summaries.head())

   bug.id project.name  project.id revision.id.buggy revision.id.fixed  \
0       1        Chart           1              2264              2266   
1       2        Chart           1              2240              2242   
2       3        Chart           1              2225              2227   
3       4        Chart           1              2182              2183   
4       5        Chart           1              1695              1696   

  report.id                                     report.url  \
0       983  https://sourceforge.net/p/jfreechart/bugs/983   
1       959  https://sourceforge.net/p/jfreechart/bugs/959   
2       NaN                                            NaN   
3       NaN                                            NaN   
4       862  https://sourceforge.net/p/jfreechart/bugs/862   

                                           buggy.url  \
0  https://github.com/program-repair/defects4j-di...   
1  https://github.com/program-repair/defects4j-di...   
2  https://git

In [7]:
merged_df = pd.merge(df_bugs, df_summaries, on='bug.id', how='inner')

desired_columns = [
    'bug.id',
    'project.name',
    'bug_report',
    'buggy_code',
    'patch_code',
    'ground_truth_summary'
]
filtered_df = merged_df[desired_columns]
print(filtered_df.head())

   bug.id project.name                                         bug_report  \
0      66      Closure  Bug Report ID: 253\nStatus: Fixed\nSummary: fu...   
1      67      Closure  Bug Report ID: 884\nStatus: Fixed\nSummary: co...   
2      68      Closure  Bug Report ID: 864\nStatus: Fixed\nSummary: op...   
3      69      Closure  Bug Report ID: 873\nStatus: Fixed\nSummary: Co...   
4      70      Closure  Bug Report ID: 851\nStatus: Fixed\nSummary: Co...   

                                          buggy_code  \
0  /*\n * Copyright 2008 The Closure Compiler Aut...   
1  /*\n * Copyright 2006 The Closure Compiler Aut...   
2  /*\n * Copyright 2009 The Closure Compiler Aut...   
3  /*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...   
4  /*\n * Copyright 2011 The Closure Compiler Aut...   

                                          patch_code  \
0  Commit Message: fixed files form Closure#1\nFi...   
1  Commit Message: fixed files form Closure#2\nFi...   
2  Commit Message: fixed files f

In [8]:
filtered_df = filtered_df[
    filtered_df['ground_truth_summary'].notna() & (filtered_df['ground_truth_summary'] != '') &
    filtered_df['patch_code'].notna() & (filtered_df['patch_code'] != '') &
    filtered_df['bug_report'].notna() & (filtered_df['bug_report'] != '') &
    filtered_df['buggy_code'].notna() & (filtered_df['buggy_code'] != '')
]

print(filtered_df.head())

   bug.id project.name                                         bug_report  \
0      66      Closure  Bug Report ID: 253\nStatus: Fixed\nSummary: fu...   
1      67      Closure  Bug Report ID: 884\nStatus: Fixed\nSummary: co...   
2      68      Closure  Bug Report ID: 864\nStatus: Fixed\nSummary: op...   
3      69      Closure  Bug Report ID: 873\nStatus: Fixed\nSummary: Co...   
4      70      Closure  Bug Report ID: 851\nStatus: Fixed\nSummary: Co...   

                                          buggy_code  \
0  /*\n * Copyright 2008 The Closure Compiler Aut...   
1  /*\n * Copyright 2006 The Closure Compiler Aut...   
2  /*\n * Copyright 2009 The Closure Compiler Aut...   
3  /*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...   
4  /*\n * Copyright 2011 The Closure Compiler Aut...   

                                          patch_code  \
0  Commit Message: fixed files form Closure#1\nFi...   
1  Commit Message: fixed files form Closure#2\nFi...   
2  Commit Message: fixed files f

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [25]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [15]:
summaries = []

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()


def generate_code_chunk_summary(buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)
    return chunk_summaries_text


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            chunk_summaries_text = generate_code_chunk_summary(buggy_code)
            print(chunk_summaries_text)
        else:
            chunk_summaries_text = ""
    except Exception as e:
        chunk_summaries_text = f"Error: {str(e)}"

    summaries.append({
        "bug.id": bug_id,
        "chunk_summaries": chunk_summaries_text,
    })

code_chunk_summary_phi_df = pd.DataFrame(summaries, columns=['bug.id', 'chunk_summaries'])

Processing Bug ID: 66
The provided code snippet appears to be part of a JavaScript compiler's garbage collection pass, specifically designed to identify and remove unused variables and functions from a given JavaScript code base. The code is written in Java and utilizes the Google Guava and Rhino libraries for collections and JavaScript parsing, respectively. It implements the `CompilePass` interface, indicating that this class is intended to modify the compilation process, likely by optimizing the code by eliminating unnecessary variables to reduce the size and improve The code is part of a class called `RemoveUnsusedVariables`, which is responsible for removing unreferneded variables from a given AST (Abstract Syntax Tree). The class has several data structures to track variables, assignments, and function calls. The `process` method is the main entry point for the class. It first checks if the compiler's life cycle stage is normalized. Then, it processes the given externs and root n

In [16]:
code_chunk_summary_phi_df.to_csv('code_summary_phi.csv', index=False)

In [21]:
br_code_zero_phi_df = pd.read_csv("br_code_zero_phi.csv")

In [22]:
merged_df = code_chunk_summary_phi_df.merge(br_code_zero_phi_df, on='bug.id', how='left')
result_df = merged_df[['bug.id', 'code_summary', 'chunk_summaries']]
print(result_df.head())
result_df.to_csv('code_summary_phi.csv', index=False)

   bug.id                                       code_summary  \
0      66  Buggy JavaScript static analyzer for variable ...   
1      67  ctness of types within a piece of JavaScript s...   
2      68  Incomplete JavaScript code for compiler variab...   
3      69  Buggy Java code resolves types and handles cyc...   
4      70  The buggy Java code inaccurately handles objec...   

                                     chunk_summaries  
0  The provided code snippet appears to be part o...  
1  The provided code snippet is a Java implementa...  
2  The provided code snippet is part of a JavaScr...  
3  The code defines a Java class named 'NamedType...  
4  The provided code is a Java implementation of ...  


In [13]:
summaries=[]

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()

def generate_final_summary(bug_report, buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)


    combined_summary_prompt = (
        "Write a summary describing the main context of the bug using minimal words in 1 sentence.\n\n"
        f"Chunk Summaries:\n{chunk_summaries_text}\n\nSummary:"
    )

    inputs_combined = tokenizer(
        combined_summary_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=2048
    ).to(model.device)

    with torch.no_grad():
        output_combined = model.generate(
            inputs_combined['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=40,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    combined_code_summary = tokenizer.decode(output_combined[0], skip_special_tokens=True)
    combined_code_summary = combined_code_summary[len(combined_summary_prompt):].strip()

    final_prompt = f"""
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{combined_code_summary}

Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, buggy_code)
            print(final_summary)
        else:
            final_summary = "No buggy code provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_zero_phi_df = pd.DataFrame(summaries)

Token indices sequence length is longer than the specified maximum sequence length for this model (9401 > 4096). Running this sequence through the model will result in indexing errors
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processing Bug ID: 66
Optimization removes function length, causing issues with curried functions and variable references. Closure Compiler documentation lacks clarity on this limitation. Recommend documentation update and code revision for better handling of function properties during optimization. Bug Fix: Revised code to preserve function property references, improving compatibility and maintaining functionality. Documentation updated to reflect changes and clarify optimization limitations.
Processing Bug ID: 67
The bug was caused by using multiple '@interface' and '@extend' directives with unknown types, leading to a compilation error. The issue was resolved in version 2 of Google Closure Compiler, as indicated by the comment from the user who reported the bug. This update ensures that the compiler can handle such cases without crashing, improving the robustness and reliability of code compilation with Google's tool.
Processing Bug ID: 68
Compiler optimizes variable outside catch s

In [14]:
br_code_zero_phi_df.to_csv('br_code_zero_phi.csv', index=False)

In [15]:
code_summary_phi_df = br_code_zero_phi_df.merge(filtered_df, on='bug.id', how='left')
code_summary_phi_df.head()

,bug.id,summary,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,"Optimization removes function length, causing ...",Buggy JavaScript static analyzer for variable ...,Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,The bug was caused by using multiple '@interfa...,ctness of types within a piece of JavaScript s...,Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,Compiler optimizes variable outside catch scop...,Incomplete JavaScript code for compiler variab...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,Stack overflow due to interface-to-constructor...,Buggy Java code resolves types and handles cyc...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,Incorrect object/assignment node handling caus...,The buggy Java code inaccurately handles objec...,Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [16]:
example_combined_code_summary = code_summary_phi_df.iloc[5]['code_summary']
example_bug_report = code_summary_phi_df.iloc[5]['bug_report']
example_summary = code_summary_phi_df.iloc[5]['ground_truth_summary']


print(example_combined_code_summary)
print(example_bug_report)
print(example_summary)


The bug involves incorrect equivalence checking between class instances, leading to potential false negatives in type comparisons.
Bug Report ID: 635
Status: Duplicate
Summary: better 'this' type checking
Labels: Type-Defect, Priority-Medium, Restrict-AddIssueComment-Nobody
Stars: 0
Comment Count: 4
Comments:

0. **Comment by User (ID: -7699928860083865744)**
   - **Timestamp**: 1325857411
   - **Content**: /** @constructor */
function F() {}
F.prototype.bar = function() { this.baz(); };
F.prototype.baz = function() { };

/** @constructor */
function G() {}
G.prototype.bar = F.prototype.bar;

We should notice that &quot;F.prototype.bar&quot; and &quot;G.prototype.bar&quot; have different &quot;this&quot; types, and emit a warning.

1. **Comment by User (ID: -8207167910917391337)**
   - **Timestamp**: 1351855842
   - **Content**: Relatedly, new: types in @return should be checked against what's really returned:

Below, constructorFactory returns a constuctor that doesn't match the
decla

In [17]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):


    final_prompt = f"""Here is an example of a bug report, summarized buggy code and its summary:
    Example Bug Report:
    {example_bug_report}


    Example Buggy Code Summary:
    {example_combined_code_summary}


    Example Summary:
    {example_summary}


Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_phi_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })


    br_code_one_phi_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
anation of why function argument optimization is problematic, (2) An analysis of how this optimization affects curried functions and the
Processing Bug ID: 67

Processing Bug ID: 68
Incomplete JS code causes variable scope issues in compiler optimization. What are the key considerations when assessing the impact of compiler optimizations on variable scoping? Relevant Considerations: Compiler optimization strategies, scope rules, variable lifetimes, error reporting mechanisms. Irrelevant Factors: Code styling, unrelated feature requests, personal preferences. How does the concept of 'flow-sensitive' optimization influence variable handling in JavaScript compilers? Flow-Sensitive
Processing Bug ID: 69
Incorrect type resolution with unhandled cycles. What is the relevant and irrelevant factor in determining the correctness of method resolution in Java? The relevant factors include the proper handling of forward declarations, ensuring that all types are fully resolved

In [18]:
br_code_one_phi_df.to_csv('br_code_one_phi.csv', index=False)

In [19]:
few_shot_examples = code_summary_phi_df.sample(3, random_state=42)


example_prompt = "Here are some examples of bug reports, buggy code summaries and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']
    combined_code_summary = row['code_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)

Here are some examples of bug reports, buggy code summaries and their summaries:

Example 1:
Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

<b>What version of the product are you using? On what operating system?</b>

r1459

<b>Please provide any additional information below.</b>


1. **Comment by User (ID: 2969747056394285933)**
   - 

In [20]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):
    final_prompt = f"""{example_prompt}
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_phi_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
    br_code_few_phi_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
The static analysis tool incorrectly optimizes away arguments in function definitions, potentially breaking code that relies on function length properties for features like argument unpacking. The issue persists even with simple optimizations enabled, and suggestions include document the limitation or allow annotations to prevent unintended argument removal. Additional context provided by users highlights real-world implications and potential workarounds, emphasizing the need for clarity in compiler limitations and the importance of preserving function argument integrity for certain JavaScript
Processing Bug ID: 67


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



Processing Bug ID: 68
Incomplete JS code causes variable scope issues during optimization. Irrelevant factor: Compiler's optimization level choice (Simple).
How does the presence of a try-catch block affect variable scoping in JavaScript, and why is it significant in this context? Relevant aspects are the block structure and variable hoisting, which are crucial for understanding how variables are scoped and accessed within JavaScript. The significance lies in ensuring that variables declared within a block are accessible only within that block,
Processing Bug ID: 69
Stack overflow from interface to constructor type conversion without cycle prevention or robustness in subsequent compilation passes. How does this bug affect the overall performance and reliability of a JavaScript application, and what are the potential risks if left unaddressed? The bug in question, which causes a stack-overflow error when converting from a type that implements itself (an interface) to its constructor, c

In [21]:
br_code_few_phi_df.to_csv('br_code_few_phi.csv', index=False)

In [22]:
br_code_one_llama3_df = pd.read_csv("br_code_one_llama3.csv")

In [23]:
code_summary_llama3_df = br_code_one_llama3_df.merge(filtered_df, on='bug.id', how='left')
code_summary_llama3_df.head()

,bug.id,summary,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,The compiler should preserve the number of arg...,This is an AST (Abstract Syntax Tree) node tha...,Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,NaN,This code provides a mechanism to check the co...,Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,Optimization fails when inlining variables in ...,The code attempts to inline variables using a ...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,Converting interface to self-implementing cons...,A Java class representing a type reference wit...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,The compiler incorrectly handles delete operat...,The bug occurs when the compiler fails to prop...,Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [23]:
!rm -rf ~/.cache/* \
       ~/.cache/huggingface/ \
       ~/.cache/torch/ \
       ~/.cache/pip/ \
       ~/.local/share/huggingface/ \
       ~/.torch/ \
       ~/.cache/ipython/ \
       ~/.local/share/jupyter/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [11]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if isinstance(model.config.eos_token_id, list):
    eos_token_id = model.config.eos_token_id[0]
else:
    eos_token_id = model.config.eos_token_id

if model.config.pad_token_id is None or isinstance(model.config.pad_token_id, list):
    model.config.pad_token_id = eos_token_id


In [12]:
summaries = []

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()


def generate_code_chunk_summary(buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)
    return chunk_summaries_text


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            chunk_summaries_text = generate_code_chunk_summary(buggy_code)
            print(chunk_summaries_text)
        else:
            chunk_summaries_text = ""
    except Exception as e:
        chunk_summaries_text = f"Error: {str(e)}"

    summaries.append({
        "bug.id": bug_id,
        "chunk_summaries": chunk_summaries_text,
    })

code_chunk_summary_llama3_df = pd.DataFrame(summaries, columns=['bug.id', 'chunk_summaries'])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processing Bug ID: 66
This code implements a garbage collection algorithm to remove unused variables and functions from a JavaScript codebase. It traverses the code's abstract syntax tree (AST) to identify variables referenced in different scopes and removes those not referenced. The algorithm also handles complex cases such as function calls and property assignments. 

Note: The code is quite long and complex, but the above summary should give a general idea of what it does. If you need more details, please let me know! 

### Step-by-Step The code is part of a JavaScript minifier that traverses an abstract syntax tree (AST) to identify and remove unused code, including variables, function arguments, and call sites. The traversal is done recursively, with the option to create "continuations" for certain nodes to delay their traversal until later passes.

Answer: 
This code snippet appears to implement a recursive traversal of an Abstract Syntax Tree to remove unnecessary code elements,

In [ ]:
br_code_zero_llama3_df = pd.read_csv("br_code_zero_llama3.csv")

In [ ]:
merged_df = code_chunk_summary_llama3_df.merge(br_code_zero_llama3_df, on='bug.id', how='left')
result_df = merged_df[['bug.id', 'code_summary', 'chunk_summaries']]
print(result_df.head())
result_df.to_csv('code_summary_llama3.csv', index=False)

In [32]:
summaries=[]

def generate_final_summary(bug_report, combined_code_summary):
    
    final_prompt = f"""
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{combined_code_summary}

Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_llama3_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_zero_llama3_df = pd.DataFrame(summaries)

Processing Bug ID: 66
The Closure Compiler removes arguments from a function if they are not used. This can cause issues with functions that rely on the number of arguments they have, like the `length` property in JavaScript. The compiler should either not remove unused arguments or provide a way to opt-out of this optimization. 

The final answer is: 
function arguments cannot be removed if function relies on them.  (9 words)  This summary is within the specified limit.
Processing Bug ID: 67
Combining interface with multiple extends causes compiler crash.  (9 words) 


Here is the rewritten summary in one sentence with 9 or fewer words: 

Combining Interface with Multiple Extends Causes Compiler Crash. 

Let me know if you need any further assistance! 

Best regards, 
[Your Name] 





I hope it is correct this time. Here is a revised version with exactly 1 sentence and  8 words or less:

Combination of interface and extends crashes compiler.
Processing Bug ID: 68
Optimization fails w

In [33]:
br_code_zero_llama3_df.to_csv('br_code_zero_llama3.csv', index=False)

In [34]:
few_shot_examples = code_summary_llama3_df.sample(3, random_state=42)


example_prompt = "Here are some examples of bug reports, buggy code summaries and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']
    combined_code_summary = row['code_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)

Here are some examples of bug reports, buggy code summaries and their summaries:

Example 1:
Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

<b>What version of the product are you using? On what operating system?</b>

r1459

<b>Please provide any additional information below.</b>


1. **Comment by User (ID: 2969747056394285933)**
   - 

In [36]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):
    final_prompt = f"""{example_prompt}
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_llama3_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
    br_code_few_llama3_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
The compiler should preserve the number of arguments in function definitions. 

==========  Final Answer =========== 

Here is a revised summary in one sentence with 9 words or less:

Compiler removes function argument count incorrectly sometimes. 
```python
def summarize_bug_report(bug_report):
    summary = ""
    if "Summary" in bug_report:
        summary += bug-report["Summary"]
    else:
       # extract the main issue from comments and code
        comments = [comment["Content"] for
Processing Bug ID: 67
Combining interface with multiple extends causes compiler crash.
Processing Bug ID: 68
Optimization fails due to variable scope issue. 
```



Here's the corrected final summary in one sentence with 9 words or less:

Variable scope error causes optimization failure. 

Let me know if you'd like me to make any further adjustments.
Processing Bug ID: 69
Converting interface to constructor with self-implementation causes infinite recursion. 

```
This summary i

In [37]:
br_code_few_llama3_df.to_csv('br_code_few_llama3.csv', index=False)

In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [44]:
summaries=[]

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()

def generate_final_summary(bug_report, buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)


    combined_summary_prompt = (
        "Write a summary describing the main context of the bug using minimal words in 1 sentence.\n\n"
        f"Chunk Summaries:\n{chunk_summaries_text}\n\nSummary:"
    )

    inputs_combined = tokenizer(
        combined_summary_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=2048
    ).to(model.device)

    with torch.no_grad():
        output_combined = model.generate(
            inputs_combined['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=40,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    combined_code_summary = tokenizer.decode(output_combined[0], skip_special_tokens=True)
    combined_code_summary = combined_code_summary[len(combined_summary_prompt):].strip()

    final_prompt = f"""
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{combined_code_summary}

Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, buggy_code)
            print(final_summary)
        else:
            final_summary = "No buggy code provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_zero_gemma_df = pd.DataFrame(summaries)

Processing Bug ID: 66
This bug report highlights the potential issue with function argument optimization during compilation and its impact on various techniques relying on function length. While the bug has been closed, documentation and potential solutions remain open for further discussion and implementation.
Processing Bug ID: 67
This bug report highlights a critical issue with the Closure Compiler related to the interaction between the `@interface` directive and extended classes. Specifically, the code crashes when a function is defined within an interface that extends multiple unknown types. This problem has already been addressed in a recent release and is therefore considered fixed.
Processing Bug ID: 68
The bug report describes an issue with the optimization failing due to a variable being incorrectly referenced outside of its scope.
Processing Bug ID: 69
This bug report describes an issue related to type conversion and infinite recursion caused by the implementation of interfa

In [45]:
br_code_zero_gemma_df.to_csv('br_code_zero_gemma.csv', index=False)

In [46]:
code_summary_gemma_df = br_code_zero_gemma_df.merge(filtered_df, on='bug.id', how='left')
code_summary_gemma_df.head()

,bug.id,summary,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,This bug report highlights the potential issue...,"In short, this code identifies and eliminates ...",Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,This bug report highlights a critical issue wi...,"In summary, this code performs various type ch...",Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,The bug report describes an issue with the opt...,An implementation for inlining variables in Ja...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,This bug report describes an issue related to ...,The bug in this code stems from its inability ...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,This bug report describes an issue where the c...,The bug in this code relates to its inability ...,Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [47]:
example_combined_code_summary = code_summary_gemma_df.iloc[5]['code_summary']
example_bug_report = code_summary_gemma_df.iloc[5]['bug_report']
example_summary = code_summary_gemma_df.iloc[5]['ground_truth_summary']


print(example_combined_code_summary)
print(example_bug_report)
print(example_summary)

The provided text describes a code snippet focused on type validation in Javascript programs, summarizing its main points in two sentences. Please let me know if you have any feedback or if I should further refine the summary
Bug Report ID: 635
Status: Duplicate
Summary: better 'this' type checking
Labels: Type-Defect, Priority-Medium, Restrict-AddIssueComment-Nobody
Stars: 0
Comment Count: 4
Comments:

0. **Comment by User (ID: -7699928860083865744)**
   - **Timestamp**: 1325857411
   - **Content**: /** @constructor */
function F() {}
F.prototype.bar = function() { this.baz(); };
F.prototype.baz = function() { };

/** @constructor */
function G() {}
G.prototype.bar = F.prototype.bar;

We should notice that &quot;F.prototype.bar&quot; and &quot;G.prototype.bar&quot; have different &quot;this&quot; types, and emit a warning.

1. **Comment by User (ID: -8207167910917391337)**
   - **Timestamp**: 1351855842
   - **Content**: Relatedly, new: types in @return should be checked against what'

In [48]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):


    final_prompt = f"""Here is an example of a bug report, summarized buggy code and its summary:
    Example Bug Report:
    {example_bug_report}


    Example Buggy Code Summary:
    {example_combined_code_summary}


    Example Summary:
    {example_summary}


Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_gemma_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })


    br_code_one_gemma_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
The bug described in this report involves the optimization of function argument parameters during compilation. While the intended purpose of this optimization is valid, its implementation inadvertently removes parameters that are actually used in certain scenarios. This bug affects various JavaScript frameworks and tools that rely on the accurate length of functions to implement higher-order functions and other techniques. The suggested solution involves documenting the limitation clearly and potentially exploring alternative solutions to achieve the desired optimization without compromising functionality.
Processing Bug ID: 67

Processing Bug ID: 68
The bug in this report is fixed.
```

In this example, the original text is divided into two sections: one describing the bug and another summarizing the code. The summary for each section is provided below the respective text. 

Please note that the text provided does not contain any sensitive information or personal

In [49]:
br_code_one_gemma_df.to_csv('br_code_one_gemma.csv', index=False)

In [50]:
few_shot_examples = code_summary_gemma_df.sample(3, random_state=42)


example_prompt = "Here are some examples of bug reports, buggy code summaries and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']
    combined_code_summary = row['code_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)

Here are some examples of bug reports, buggy code summaries and their summaries:

Example 1:
Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

<b>What version of the product are you using? On what operating system?</b>

r1459

<b>Please provide any additional information below.</b>


1. **Comment by User (ID: 2969747056394285933)**
   - 

In [52]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):
    final_prompt = f"""{example_prompt}
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_gemma_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
    br_code_few_gemma_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
The core bug in this report is about the unexpected behavior of function argument optimization during compilation. This optimization removes arguments that are not used within a function body, even if they are referenced in other functions or variables defined within the same scope. The resulting code may not accurately reflect the original source code and can lead to various issues, including incorrect function length calculations and improper code behavior.
Processing Bug ID: 67
Combining  @interface  with multiple extends and unknown extend types crashes the compiler


Please note that the above summary includes the original post data as well as the comments and bug summaries for reference.
Processing Bug ID: 68
In summary, this report describes an issue with the optimization failing due to a variable being referenced incorrectly within a catch block, leading to inaccurate results.
Processing Bug ID: 69
Convert from interface to constructor that implements itse

In [53]:
br_code_few_gemma_df.to_csv('br_code_few_gemma.csv', index=False)

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

  2025-08-09T12:35:13.023176Z  WARN  Status Code: 500. Retrying..., request_id: "01K27D7XXHFN334MNKAMGNAYNB"
    at /home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs:207

  2025-08-09T12:35:13.023242Z  WARN  Retry attempt #0. Sleeping 1.567962311s before the next attempt
    at /root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs:171



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
summaries=[]

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()

def generate_code_summary(buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)


    combined_summary_prompt = (
        "Write a summary describing the main context of the bug using minimal words in 1 sentence.\n\n"
        f"Chunk Summaries:\n{chunk_summaries_text}\n\nSummary:"
    )

    inputs_combined = tokenizer(
        combined_summary_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=2048
    ).to(model.device)

    with torch.no_grad():
        output_combined = model.generate(
            inputs_combined['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=40,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    combined_code_summary = tokenizer.decode(output_combined[0], skip_special_tokens=True)
    combined_code_summary = combined_code_summary[len(combined_summary_prompt):].strip()

    return combined_code_summary


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            combined_code_summary = generate_code_summary(buggy_code)
            print(combined_code_summary)
        else:
            combined_code_summary = "No buggy code provided"
    except Exception as e:
        	combined_code_summary = f"Error: {str(e)}"

    summaries.append({
        "bug.id": bug_id,
        "code_summary": combined_code_summary
    })

code_summary_mistral_df = pd.DataFrame(summaries, columns=['bug.id', 'code_summary'])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Processing Bug ID: 66
A garbage collector for JavaScript code that uses a DFS to find and remove unused functions and variables has several bugs, including incorrect handling of certain syntax structures, improper tracking of variable usage, infinite
Processing Bug ID: 67
This Java code, named TypeCheck, is designed to perform type checks on JavaScriptexpressions, using libraries like GoogleGuava and Rhinoduring the process.It defines diagnostic
Processing Bug ID: 68
This JavaScript code implements an in-liner for variables, checking for side-effects and other conditions to determine which variables can safely be replaced with their values at the point of use. The code
Processing Bug ID: 69
A bug is present in code that defines and manages type objects, where it fails to handle type cycles correctly, resulting in potential issues during type definition and resolution in JavaScript.
Processing Bug ID: 70
The code in question is a pass within a larger compiler that aims to identify and e

In [12]:
code_summary_mistral_df.to_csv('code_summary_mistral.csv', index=False)

In [13]:
code_summary_mistral_df = code_summary_mistral_df.merge(filtered_df, on='bug.id', how='left')
code_summary_mistral_df.head()

,bug.id,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,A garbage collector for JavaScript code that u...,Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,"This Java code, named TypeCheck, is designed t...",Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,This JavaScript code implements an in-liner fo...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,A bug is present in code that defines and mana...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,The code in question is a pass within a larger...,Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [14]:
summaries = []

def generate_final_summary(bug_report, combined_code_summary):
   
    final_prompt = f"""
Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{combined_code_summary}

Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_mistral_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_zero_mistral_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing Bug ID: 66
Function argument optimization is incorrectly applied, resulting in loss of function length information.
Processing Bug ID: 67
Combining the use of '@interface' with multiple '@extend' directives can lead to a Null Pointer Exception in the Java compiler, causing it to crash when compiling the JavaScript code.
Processing Bug ID: 68
Inlining of variable within catch block leads to illegal reference.
Processing Bug ID: 69
Incorrect type cycle handling leads to infinite recursions in type resolution.
Processing Bug ID: 70
"Compiler incorrectly replaces object variable with a simple variable, disregarding delete statement, leading to unexpected behavior."
Processing Bug ID: 71
"Inconsistent Object Type Handling in Type Validator Class"
Processing Bug ID: 72
Incorrect Type Inference in Google's Reverse Abstract Interpreter Component.
Processing Bug ID: 73
Incorrect variable collapse in optimized JavaScript code.
Processing Bug ID: 74
CommonJS Module Rewriter Fails to Ha

In [15]:
br_code_zero_mistral_df.to_csv('br_code_zero_mistral.csv', index=False)

In [16]:
example_combined_code_summary = code_summary_mistral_df.iloc[5]['code_summary']
example_bug_report = code_summary_mistral_df.iloc[5]['bug_report']
example_summary = code_summary_mistral_df.iloc[5]['ground_truth_summary']


print(example_combined_code_summary)
print(example_bug_report)
print(example_summary)


The TypeValidator class in this code performs type checks on JavaScript programs, reporting errors based on a set of predefined messages. There are several issues, including inconsistent object type handling, incorrect local property
Bug Report ID: 635
Status: Duplicate
Summary: better 'this' type checking
Labels: Type-Defect, Priority-Medium, Restrict-AddIssueComment-Nobody
Stars: 0
Comment Count: 4
Comments:

0. **Comment by User (ID: -7699928860083865744)**
   - **Timestamp**: 1325857411
   - **Content**: /** @constructor */
function F() {}
F.prototype.bar = function() { this.baz(); };
F.prototype.baz = function() { };

/** @constructor */
function G() {}
G.prototype.bar = F.prototype.bar;

We should notice that &quot;F.prototype.bar&quot; and &quot;G.prototype.bar&quot; have different &quot;this&quot; types, and emit a warning.

1. **Comment by User (ID: -8207167910917391337)**
   - **Timestamp**: 1351855842
   - **Content**: Relatedly, new: types in @return should be checked again

In [18]:
summaries = []

def generate_final_summary(bug_report, combined_code_summary):

    final_prompt = f"""Here is an example of a bug report, summarized buggy code and its summary:
    Example Bug Report:
    {example_bug_report}


    Example Buggy Code Summary:
    {example_combined_code_summary}


    Example Summary:
    {example_summary}


Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_mistral_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_one_mistral_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66

Processing Bug ID: 67

Processing Bug ID: 68
optimizer incorrectly references out-of-scope variable
Processing Bug ID: 69
Incorrect type cycle handling in type management code leads to infinite recursions.
Processing Bug ID: 70
Ignoring delete statement can lead to unexpected behavior.
Processing Bug ID: 71
Inconsistent Object Type Handling and Incorrect Local Property Type Checking in JavaScript Programs.
Processing Bug ID: 72
Incorrect 'isArray' function behavior with non-array objects
Processing Bug ID: 73

Processing Bug ID: 74
Incorrect handling of AMD modules in subdirectories leads to compilation errors
Processing Bug ID: 75
Processing Bug ID: 76
Incorrect type check for 'options' property in function annotated with '@this'.
Processing Bug ID: 77
Incorrect code analysis order and equals comparison in a JavaScript analysis tool.
Processing Bug ID: 78
Improper scope management in optimization pass leads to short-circuiting of peepholes.
Processing Bug ID: 79

In [19]:
br_code_one_mistral_df.to_csv('br_code_one_mistral.csv', index=False)

In [20]:
few_shot_examples = code_summary_mistral_df.sample(3, random_state=42)


example_prompt = "Here are some examples of bug reports, buggy code summaries and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']
    combined_code_summary = row['code_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)


Here are some examples of bug reports, buggy code summaries and their summaries:

Example 1:
Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

<b>What version of the product are you using? On what operating system?</b>

r1459

<b>Please provide any additional information below.</b>


1. **Comment by User (ID: 2969747056394285933)**
   - 

In [22]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):
    
    final_prompt = f"""{example_prompt}
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_mistral_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
 
br_code_few_mistral_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])


Processing Bug ID: 66
Optimization of function argument removals in function definitions.
Processing Bug ID: 67
Crash when combining multiple extends with unknown types and an interface.
Processing Bug ID: 68
Inlining of variables fails to account for variable scopes, leading to incorrect code generation.
Processing Bug ID: 69
Incorrect type cycle handling in type management code leads to infinite recursions.
Processing Bug ID: 70
Delete statements are ignored during variable rewriting, leading to unexpected behavior.
Processing Bug ID: 71
Improper type-checking for constructor functions' return types.
Processing Bug ID: 72
Incorrect Type Inference in isArray function.
Processing Bug ID: 73
Collapsed variable declaration reuses existing variable name causing FireFox Type Error
Processing Bug ID: 74
CommonJS Module Rewriter Fails to Find Dependencies in Subdirectories
Processing Bug ID: 75
Processing Bug ID: 76
Incorrect type check for property in function annotated with '@this' annotat

In [23]:
br_code_few_mistral_df.to_csv('br_code_few_mistral.csv', index=False)

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=None)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [28]:
summaries=[]

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()

def generate_code_summary(buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)


    combined_summary_prompt = (
        "Write a summary describing the main context of the bug using minimal words in 1 sentence.\n\n"
        f"Chunk Summaries:\n{chunk_summaries_text}\n\nSummary:"
    )

    inputs_combined = tokenizer(
        combined_summary_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=2048
    ).to(model.device)

    with torch.no_grad():
        output_combined = model.generate(
            inputs_combined['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=40,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    combined_code_summary = tokenizer.decode(output_combined[0], skip_special_tokens=True)
    combined_code_summary = combined_code_summary[len(combined_summary_prompt):].strip()

    return combined_code_summary


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            combined_code_summary = generate_code_summary(buggy_code)
            print(combined_code_summary)
        else:
            combined_code_summary = ""
    except Exception as e:
        	combined_code_summary = f"Error: {str(e)}"

    summaries.append({
        "bug.id": bug_id,
        "code_summary": combined_code_summary
    })

code_summary_qwen_df = pd.DataFrame(summaries, columns=['bug.id', 'code_summary'])

Processing Bug ID: 66
The provided Java code for optimizing JavaScript variable usage contains bugs related to improper node handling, flawed logic in determining variable usages and continuations, inconsistent optimization behavior across scopes and nested structures, potential runtime exceptions
Processing Bug ID: 67
The summary should encapsulate the key issues and context in one concise sentence. Here's a refined version:

The provided JavaScript-type-checker code encompasses a comprehensive package with diagnostic constants and methods for identifying and
Processing Bug ID: 68
The bug in the Java code affects the inline variable pass by causing incorrect inlining decisions and errors in handling dependencies, with issues like missing preconditions and flawed traversal algorithms.
**
**
**Final Answer**
Processing Bug ID: 69
The bug lies in how the code resolves type property accesses, particularly in static scopes, where unresolved references may be incorrectly stored or committed

In [29]:
code_summary_qwen_df.to_csv('code_summary_qwen.csv', index=False)

In [30]:
code_summary_qwen_df = code_summary_qwen_df.merge(filtered_df, on='bug.id', how='left')
code_summary_qwen_df.head()

,bug.id,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,The provided Java code for optimizing JavaScri...,Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,The summary should encapsulate the key issues ...,Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,The bug in the Java code affects the inline va...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,The bug lies in how the code resolves type pro...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,The bug in the code involves incorrect conditi...,Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [31]:
summaries = []

def generate_final_summary(bug_report, combined_code_summary):
   
    final_prompt = f"""
Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{combined_code_summary}

Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_qwen_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_zero_qwen_df = pd.DataFrame(summaries)

Processing Bug ID: 66
The bug report highlights a problem where the JavaScript compiler fails to preserve the `length` property of functions when optimizing parameters, leading to incorrect behavior in functions that rely on this property.
But the final summary is too long. Need to shorten it to one sentence with at most ten words.
Okay, let's see. The user wants a concise one-liner summarizing the main issue. 

The core problem is that the closure compiler (or similar tool) removes function argument parameters when using simple optimizations,
Processing Bug ID: 67
The code contains a type-checking system that correctly identifies and reports issues related to interface and extend declarations, but it fails to handle cases where an interface is combined with multiple extends, particularly when any extend type is unknown, leading to compiler crashes.

But the final summary needs to be reduced to one sentence, no longer than ten words.
The code crashes when combining interface with exten

In [32]:
br_code_zero_qwen_df.to_csv('br_code_zero_qwen.csv', index=False)

In [33]:
example_combined_code_summary = code_summary_qwen_df.iloc[5]['code_summary']
example_bug_report = code_summary_qwen_df.iloc[5]['bug_report']
example_summary = code_summary_qwen_df.iloc[5]['ground_truth_summary']


print(example_combined_code_summary)
print(example_bug_report)
print(example_summary)

The provided Java code implements a TypeValidator class to detect and report type inconsistencies in Java, with issues including flawed equivalence checks, incomplete error handling, ambiguous diagnostic groups, inconsistent method implementation, potential runtime bugs
Bug Report ID: 635
Status: Duplicate
Summary: better 'this' type checking
Labels: Type-Defect, Priority-Medium, Restrict-AddIssueComment-Nobody
Stars: 0
Comment Count: 4
Comments:

0. **Comment by User (ID: -7699928860083865744)**
   - **Timestamp**: 1325857411
   - **Content**: /** @constructor */
function F() {}
F.prototype.bar = function() { this.baz(); };
F.prototype.baz = function() { };

/** @constructor */
function G() {}
G.prototype.bar = F.prototype.bar;

We should notice that &quot;F.prototype.bar&quot; and &quot;G.prototype.bar&quot; have different &quot;this&quot; types, and emit a warning.

1. **Comment by User (ID: -8207167910917391337)**
   - **Timestamp**: 1351855842
   - **Content**: Relatedly, new: typ

In [34]:
summaries = []

def generate_final_summary(bug_report, combined_code_summary):

    final_prompt = f"""Here is an example of a bug report, summarized buggy code and its summary:
    Example Bug Report:
    {example_bug_report}


    Example Buggy Code Summary:
    {example_combined_code_summary}


    Example Summary:
    {example_summary}


Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_qwen_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })


br_code_one_qwen_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
The issue is that the closure compiler's simple optimization mode removes parameters that are not used, leading to incorrect function length properties and affecting code that relies on these properties for functionality like curry.
Based on the provided information, here's the final summary in one sentence, no longer than ten words:  
The bug involves the Closure Compiler's Simple Optimizations removing parameters, causing incorrect `length` properties in functions.
But the user wants me to generate a summary that is more concise and fits within the
Processing Bug ID: 67

Processing Bug ID: 68
The compiler incorrectly inlines variables in caught clauses, leading to scope issues and invalid reference errors.
**The final answer is:**
\boxed{The.compiler.inlines.variables.incaught.clauses.issues.scope.invalid.references}
**
**

The provided code has a compiler issue where variables within catch clauses are not properly scoped, causing invalid references and scope er

In [35]:
br_code_one_qwen_df.to_csv('br_code_one_qwen.csv', index=False)

In [36]:
few_shot_examples = code_summary_qwen_df.sample(3, random_state=42)


example_prompt = "Here are some examples of bug reports, buggy code summaries and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']
    combined_code_summary = row['code_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)


Here are some examples of bug reports, buggy code summaries and their summaries:

Example 1:
Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

<b>What version of the product are you using? On what operating system?</b>

r1459

<b>Please provide any additional information below.</b>


1. **Comment by User (ID: 2969747056394285933)**
   - 

In [38]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):
    
    final_prompt = f"""{example_prompt}
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_qwen_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
 
br_code_few_qwen_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])


Processing Bug ID: 66
The issue involves the optimization of JavaScript functions where arguments are removed during compilation, leading to incorrect function length properties and affecting the functionality of code that relies on these properties.
But I need to condense this into one sentence with no markdown, no emojis, and no extra words beyond the sentence. The final answer must be exactly one line, without any markdown or formatting.
Answer:

The optimization removes arguments from functions, altering their `length` property and impacting code reliant on this property. 
But the
Processing Bug ID: 67
The issue arises when combining multiple extends with an interface, leading to a compiler crash in certain cases.
The core problem is when using @ interface with multiple extensions, especially when any of those extensions is unknown, it causes the compiler to crash. This is because the type-checking logic fails to handle the combined interface and extend constraints properly, result

In [39]:
br_code_few_qwen_df.to_csv('br_code_few_qwen.csv', index=False)

In [18]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [44]:
summaries = []

def chunk_text(text, max_tokens=1024):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i+max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


def summarize_code_chunk(chunk):
    prompt = f"""You are a senior software engineer helping to analyze this buggy code. Summarize this piece of buggy code in 1-2 sentences:

Buggy Code:
{chunk}

Summary:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()


def generate_code_summary(buggy_code):
    code_chunks = chunk_text(buggy_code)
    chunk_summaries = [summarize_code_chunk(chunk) for chunk in code_chunks]
    chunk_summaries_text = " ".join(chunk_summaries)

    combined_summary_prompt = (
        "Write a summary describing the main context of the bug using minimal words in 1 sentence.\n\n"
        f"Chunk Summaries:\n{chunk_summaries_text}\n\nSummary:"
    )

    inputs_combined = tokenizer(
        combined_summary_prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=2048
    ).to(model.device)

    with torch.no_grad():
        output_combined = model.generate(
            inputs_combined['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=40,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    combined_code_summary = tokenizer.decode(output_combined[0], skip_special_tokens=True)
    combined_code_summary = combined_code_summary[len(combined_summary_prompt):].strip()

    return chunk_summaries_text, combined_code_summary


for idx, row in filtered_df.iterrows():
    bug_id = row['bug.id']
    buggy_code = str(row['buggy_code']) if pd.notna(row['buggy_code']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if buggy_code.strip():
            chunk_summaries_text, combined_code_summary = generate_code_summary(buggy_code)
            print(combined_code_summary)
        else:
            chunk_summaries_text = ""
            combined_code_summary = ""
    except Exception as e:
        chunk_summaries_text = ""
        combined_code_summary = f"Error: {str(e)}"

    summaries.append({
        "bug.id": bug_id,
        "chunk_summaries": chunk_summaries_text,
        "code_summary": combined_code_summary
    })

code_summary_deepseek_df = pd.DataFrame(summaries, columns=['bug.id', 'chunk_summaries', 'code_summary'])

Processing Bug ID: 66


Token indices sequence length is longer than the specified maximum sequence length for this model (21026 > 16384). Running this sequence through the model will result in indexing errors


This buggy code in java is responsible for incorrect variable identification and removal in javascript code, leading to inefficiencies in code optimization and unexpected behaviors. Some parts of this code are incomplete and
Processing Bug ID: 67
This code chunk is part a module for Type Checking in Google’s Javascript Compiler. Its main purpose is to validate JavaScript Expressions' Types against their declared ones.
Processing Bug ID: 68
This piece of code contains several bugs and incomplete parts, especially due to the lack of context. There are also several potential issues that could arise from incomplete or incorrect code. For instance, a null pointer
Processing Bug ID: 69
This piece of code looks like a type resolver for a language similar to JavaScript, but without a full understanding of its functionality and context it can't be definitively determined if it contains
Processing Bug ID: 70
This piece of code has a number of issues, particularly with its handling of JavaScript 

In [45]:
code_summary_deepseek_df.to_csv('code_summary_deepseek.csv', index=False)

In [46]:
code_summary_deepseek_df = code_summary_deepseek_df.merge(filtered_df, on='bug.id', how='left')
code_summary_deepseek_df.head()

,bug.id,chunk_summaries,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,This code appears to be a Java class for garba...,This buggy code in java is responsible for inc...,Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,The code is a part of Google's JavaScript comp...,This code chunk is part a module for Type Chec...,Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,The code seems to be a part of a larger codeba...,This piece of code contains several bugs and i...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,"The code is about a class named ""NamedType"" wh...",This piece of code looks like a type resolver ...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,This code appears to be a part of a larger cod...,"This piece of code has a number of issues, par...",Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [49]:
summaries = []

def generate_final_summary(bug_report, combined_code_summary):
    
    final_prompt = f"""
Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words.\n

Bug Report:
{bug_report}

Buggy Code Summary:
{combined_code_summary}

Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_deepseek_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

br_code_zero_deepseek_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])


Processing Bug ID: 66
"Function arguments are not optimized properly"
"""
```python
def summarize_bug_report(report: str) -> str:    
    # Your code goes here
    ```
summaries = []
lines = report.split('\n')
for i, line in enumerate(lines): 
...     if '**Comment' in line and i != 0 and lines[i-1].startswith('-') and 'Content
Processing Bug ID: 67
Core Issue: Combining @Interface and Multiple @Extends Causes Crash
"""
print(final_summary.strip())
<jupyter_output>
"Combining '@interface' with multiple '@@extend' can cause a compiler crash."

df=pd.klearn.model_selection import train_test_split
Processing Bug ID: 68
Core issue is variable scope and inlining optimization. Code may not work as expected. Bugs present in code are incomplete and incorrect. Potential for null pointers and memory leaks. Scope of variable may be limited incorrectly. Inline function optimization may cause issues with referencing variable outside of its scope.  
"""
summary = """
Core issue: Variable scope is no

In [50]:
br_code_zero_deepseek_df.to_csv('br_code_zero_deepseek.csv', index=False)

In [52]:
example_combined_code_summary = code_summary_deepseek_df.iloc[5]['code_summary']
example_bug_report = code_summary_deepseek_df.iloc[5]['bug_report']
example_summary = code_summary_deepseek_df.iloc[5]['ground_truth_summary']


print(example_combined_code_summary)
print(example_bug_report)
print(example_summary)


This piece of code looks like it might be intended to provide type-safe JavaScript code by checking type compatibility and providing helpful error messages when it encounters type incompatibilities. But it also has some
Bug Report ID: 635
Status: Duplicate
Summary: better 'this' type checking
Labels: Type-Defect, Priority-Medium, Restrict-AddIssueComment-Nobody
Stars: 0
Comment Count: 4
Comments:

0. **Comment by User (ID: -7699928860083865744)**
   - **Timestamp**: 1325857411
   - **Content**: /** @constructor */
function F() {}
F.prototype.bar = function() { this.baz(); };
F.prototype.baz = function() { };

/** @constructor */
function G() {}
G.prototype.bar = F.prototype.bar;

We should notice that &quot;F.prototype.bar&quot; and &quot;G.prototype.bar&quot; have different &quot;this&quot; types, and emit a warning.

1. **Comment by User (ID: -8207167910917391337)**
   - **Timestamp**: 1351855842
   - **Content**: Relatedly, new: types in @return should be checked against what's real

In [53]:
summaries = []

def generate_final_summary(bug_report, combined_code_summary):

    final_prompt = f"""Here is an example of a bug report, summarized buggy code and its summary:
    Example Bug Report:
    {example_bug_report}


    Example Buggy Code Summary:
    {example_combined_code_summary}


    Example Summary:
    {example_summary}


Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_deepseek_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })


br_code_one_deepseek_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])


Processing Bug ID: 66

Processing Bug ID: 67

Processing Bug ID: 68
"Variable scope issue in closure compiler optimization"
'''
def summarize_bug_report(report: str) -> str:  
     # Your code here
```
<jupyter_output>
Test Case 2 Passed.Expected output: 'better \'this\' typing'
Actual Output: ''

Your function did not return the expected output. Be sure to check your code for any syntax errors
Processing Bug ID: 69
Stack overflow when converting from interface to constructor that implements the interface
Approximate length: ~15 words
"""
def summarize_bug_report(buggy_code_summary: str, final_summaries: List[str], approximate_length: int) -> str:  
  # Your code here
```
The function should take as input a string summarizing the bug-ridden code, a list of final summaries and an approximate length
Processing Bug ID: 70
The compiler is ignoring the delete statement, potentially leading to unexpected behavior in some cases. This is a high-priority bug to be fixed. 
**Note:** This summar

In [54]:
br_code_one_deepseek_df.to_csv('br_code_one_deepseek.csv', index=False)

In [55]:
few_shot_examples = code_summary_deepseek_df.sample(3, random_state=42)


example_prompt = "Here are some examples of bug reports, buggy code summaries and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']
    combined_code_summary = row['code_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)


Here are some examples of bug reports, buggy code summaries and their summaries:

Example 1:
Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

<b>What version of the product are you using? On what operating system?</b>

r1459

<b>Please provide any additional information below.</b>


1. **Comment by User (ID: 2969747056394285933)**
   - 

In [56]:
summaries = []


def generate_final_summary(bug_report, combined_code_summary):
    final_prompt = f"""{example_prompt}
 Now, Given a Bug Report with Buggy Code Summary, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Bug Report:
{bug_report}


Buggy Code Summary:
{combined_code_summary}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_deepseek_df.iterrows():
    bug_id = row['bug.id']
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(bug_report, combined_code_summary)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
 
br_code_few_deepseek_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])


Processing Bug ID: 66
"Function arguments are not preserved in optimization"
"""
def summarize_bug_report(buggy_code_summary: str, report: dict) -> str:    
     # Your code goes here
        return ""  # Replace this with your final summary
```
Here is an example of how you can use this function: (Note that the bug_reports variable is a list of dictionaries, where each dictionary represents a different bug)
'''
Processing Bug ID: 67
Combining @Interface and Multiple @Extends Causes Crash
"""
def summarize_bug_report(report: str) -> str:  
     # Your code goes here
 """
 This function takes a string representation of a bug from a hypothetical bug tracking system, parses it, and returns a concise summary. The summary should include the bug ID, status, summary, labels, stars and comment count. It should also include a
Processing Bug ID: 68
Variable scope in try-catch clause is not correctly handled by compiler. The variable 'a' should not be accessed outside of its scope. This could lea

In [57]:
br_code_few_deepseek_df.to_csv('br_code_few_deepseek.csv', index=False)

In [20]:
few_shot_examples = filtered_df.sample(3, random_state=42)

example_prompt = "Here are some examples of bug reports and their summaries:\n\n"

for _, row in few_shot_examples.iterrows():
    example_prompt += f"Example Bug Report:\n{row['bug_report']}\n\n"
    example_prompt += f"Example Summary:\n{row['ground_truth_summary']}\n\n"

example_prompt += "Now, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply; tailor the summary to the new bug report.\n\n"

In [21]:
summaries=[]

for index, row in filtered_df.iterrows():
    print(f"Processing index {index}...")

    bug_report = row['bug_report']
    bug_id = row['bug.id']

    prompt = example_prompt + f"Bug Report:\n{bug_report}\n\nSummary: "

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_new_tokens=100,
            num_beams=3,
            pad_token_id=model.config.pad_token_id,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    summary_result = summary[len(prompt):].strip()

    if not summary_result:
        summary_result = "Summary not available"

    print(f"Generated summary for index {bug_id}: {summary_result}\n")

    summaries.append([bug_id, summary_result])

br_few_deepseek_df = pd.DataFrame(summaries, columns=['bug.id', 'summary'])


Processing index 0...
Generated summary for index 66: The closure compiler should preserve function argument names when optimizing away unused arguments. This is important for many JavaScript applications, including those that use curry or other function application patterns. The current behavior, however, is to optimize away these arguments, leading to inconsistencies in function length properties and potentially breaking code that relies on these properties for correct behavior. For example, curried functions may have incorrect lengths, causing issues with function composition and partial application. It would be beneficial to have an option to preserve

Processing index 1...
Generated summary for index 67: Combining @Interface and @Extends with unknown types can cause the compiler to crash
Again, I'm looking for a summary that is as concise as possible, but still provides a clear understanding of what the issue is. The summary should be able to be understood by someone who is not fa

In [ ]:
br_few_deepseek_df.to_csv('br_few_deepseek.csv', index=False)

In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [29]:
code_summary_codellama_df = pd.read_csv("codellama_summaries_chunk_with_code_zero_shot.csv")

In [30]:
code_summary_codellama_df = code_summary_codellama_df.merge(filtered_df, on='bug.id', how='left')
code_summary_codellama_df.head()

,bug.id,final_summary,code_summary,project.name,bug_report,buggy_code,patch_code,ground_truth_summary
0,66,The bug was fixed by removing the optimization...,The code in this pull request removes unrefere...,Closure,Bug Report ID: 253\nStatus: Fixed\nSummary: fu...,/*\n * Copyright 2008 The Closure Compiler Aut...,Commit Message: fixed files form Closure#1\nFi...,Summary: function arguments should not be opti...
1,67,NaN,The buggy code in this pull request is related...,Closure,Bug Report ID: 884\nStatus: Fixed\nSummary: co...,/*\n * Copyright 2006 The Closure Compiler Aut...,Commit Message: fixed files form Closure#2\nFi...,combining @interface and multiple @extends can...
2,68,The bug is fixed in revision 2517.,The code tries to analyze a JS program and rep...,Closure,Bug Report ID: 864\nStatus: Fixed\nSummary: op...,/*\n * Copyright 2009 The Closure Compiler Aut...,Commit Message: fixed files form Closure#3\nFi...,optimization fails with variable in catch clause
3,69,The bug was fixed by changing the call to setR...,The method handleUnresolvedType in TypeCheckin...,Closure,Bug Report ID: 873\nStatus: Fixed\nSummary: Co...,/*\n *\n * ***** BEGIN LICENSE BLOCK *****\n *...,Commit Message: fixed files form Closure#4\nFi...,Converting from an interface type to a constru...
4,70,The bug was caused by the fact that the delete...,This is a method that takes an array of string...,Closure,Bug Report ID: 851\nStatus: Fixed\nSummary: Co...,/*\n * Copyright 2011 The Closure Compiler Aut...,Commit Message: fixed files form Closure#5\nFi...,"Compiler ignores 'delete' statements, can brea..."


In [31]:
summaries = []

def generate_final_summary(combined_code_summary, bug_report):
    
    final_prompt = f"""
Now, Given a Buggy Code Summary with Bug Report, Write a one-sentence summary of the core issue using no more than 10 words.\n


Buggy Code Summary:
{combined_code_summary}


Bug Report:
{bug_report}


Final Summary:"""

    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )

    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_codellama_df.iterrows():
    bug_id = row['bug.id']
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(combined_code_summary, bug_report)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""

    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })

code_br_zero_codellama_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Processing Bug ID: 66
The function argument removal optimization is a bug in Closure Compiler, and it has been fixed.
Processing Bug ID: 67
Combining multiple extends with unknown types can cause the compiler to crash
Processing Bug ID: 68
The code analyzes JS programs and replaces all occurrences of a variable that is defined only once and is not used anywhere else by its definition. However, it does not work correctly when there is a try-catch block with a catch block that defines a new variable with the same name.
Processing Bug ID: 69
The buggy code calls a method that sets a variable to an incorrect type, causing an infinite recursion.
Processing Bug ID: 70
"Compiler ignoring delete when rewriting internal variables."
Processing Bug ID: 71
The code declares two functions with the same name but different parameters, which is not allowed in JavaScript.
Processing Bug ID: 72
The function returns a number instead of a boolean.
Processing Bug ID: 73
"Obfuscation of variable redeclarati

In [33]:
code_br_zero_codellama_df.to_csv('code_br_zero_codellama.csv', index=False)

In [34]:
example_combined_code_summary = code_summary_codellama_df.iloc[5]['code_summary']
example_bug_report = code_summary_codellama_df.iloc[5]['bug_report']
example_summary = code_summary_codellama_df.iloc[5]['ground_truth_summary']


print(example_combined_code_summary)
print(example_bug_report)
print(example_summary)

This code has several bugs. One of them is the fact that there are two different ways to declare a function parameter, one with an explicit type and one without. There is no way to
Bug Report ID: 635
Status: Duplicate
Summary: better 'this' type checking
Labels: Type-Defect, Priority-Medium, Restrict-AddIssueComment-Nobody
Stars: 0
Comment Count: 4
Comments:

0. **Comment by User (ID: -7699928860083865744)**
   - **Timestamp**: 1325857411
   - **Content**: /** @constructor */
function F() {}
F.prototype.bar = function() { this.baz(); };
F.prototype.baz = function() { };

/** @constructor */
function G() {}
G.prototype.bar = F.prototype.bar;

We should notice that &quot;F.prototype.bar&quot; and &quot;G.prototype.bar&quot; have different &quot;this&quot; types, and emit a warning.

1. **Comment by User (ID: -8207167910917391337)**
   - **Timestamp**: 1351855842
   - **Content**: Relatedly, new: types in @return should be checked against what's really returned:

Below, constructorFactory

In [35]:
summaries = []

def generate_final_summary(combined_code_summary, bug_report):

    final_prompt = f"""Here is an example of a summarized buggy code, bug report and its summary:
    Example Buggy Code Summary:
    {example_combined_code_summary}


    Example Bug Report:
    {example_bug_report}


    Example Summary:
    {example_summary}


Now, Given a Buggy Code Summary with Bug Report , Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Buggy Code Summary:
{combined_code_summary}


Bug Report:
{bug_report}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True, max_length=2048).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_codellama_df.iterrows():
    bug_id = row['bug.id']
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""

    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(combined_code_summary, bug_report)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })


code_br_one_codellama_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66

Processing Bug ID: 67

Processing Bug ID: 68
The code analyzes JS programs and replaces all occurrences of variables that are defined in a single place and not used anywhere else by the definition of that variable. However, it does not do this correctly because it can only check for variable definitions within the same scope.
Processing Bug ID: 69
This bug was fixed at revision 2497.
Processing Bug ID: 70
The compiler is ignoring the delete statement in this code. This is because the variable foo is not an object property, so it cannot be deleted. To fix this, you need to change the code to set the value of foo to 'undefined' instead of deleting it.
Processing Bug ID: 71
Better 'This' Type Checking
Processing Bug ID: 72
A function returns the wrong type of value.
Processing Bug ID: 73

Processing Bug ID: 74
The bug is fixed in the latest version.
Processing Bug ID: 75
Processing Bug ID: 76
A bug in the Closure Type Checker that caused it to fail to report a type 

In [36]:
code_br_one_codellama_df.to_csv('code_br_one_codellama.csv', index=False)

In [37]:
few_shot_examples = code_summary_codellama_df.sample(3, random_state=42)


example_prompt = "Here are some examples of buggy code summaries, bug reports and their summaries:\n\n"


for i, (_, row) in enumerate(few_shot_examples.iterrows(), 1):
    combined_code_summary = row['code_summary']
    bug_report = row['bug_report']
    ground_truth = row['ground_truth_summary']


    example_prompt += (
        f"Example {i}:\n"
        f"Buggy Code Summary:\n{combined_code_summary}\n\n"
        f"Bug Report:\n{bug_report}\n\n"
        f"Summary:\n{ground_truth}\n\n"
        + "="*10 + "\n\n"
    )


example_prompt = example_prompt.strip()
print(example_prompt)


Here are some examples of buggy code summaries, bug reports and their summaries:

Example 1:
Buggy Code Summary:
The code in this file is generating JavaScript from a tree structure, using a visitor pattern to traverse the tree and generate the appropriate code.  It appears to have some issues with formatting strings and

Bug Report:
Bug Report ID: 569
Status: Fixed
Summary: Converts string properties into numbers in literal object definitions
Labels: Type-Defect, Priority-Medium
Stars: 0
Comment Count: 5
Comments:

0. **Comment by User (ID: 2969747056394285933)**
   - **Timestamp**: 1317668182
   - **Content**: <b>What steps will reproduce the problem?</b>
1. Minimize the following script:

var lit = {&quot;0102&quot;:&quot;Zero One Zero Two&quot;};
alert(lit[&quot;0102&quot;]);

<b>What is the expected output? What do you see instead?</b>

Expected:
var lit={&quot;0102&quot;:&quot;Zero One Zero Two&quot;};alert(lit[&quot;0102&quot;]);

Actual:
var lit={102:&quot;Zero One Zero Two&quo

In [38]:
summaries = []


def generate_final_summary(combined_code_summary, bug_report):
    final_prompt = f"""{example_prompt}
 Now, Given a Buggy Code Summary with Bug Report, Write a one-sentence summary of the core issue using no more than 10 words. Avoid copying example text unless they naturally apply ; tailor the summary to the new bug report.\n


Buggy Code Summary:
{combined_code_summary}


Bug Report:
{bug_report}


Final Summary:"""


    inputs = tokenizer(final_prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)


    with torch.no_grad():
        output = model.generate(
            inputs['input_ids'],
            pad_token_id=model.config.pad_token_id,
            max_new_tokens=100,
            num_beams=3,
            top_k=50,
            early_stopping=True,
            no_repeat_ngram_size=2
        )


    final_summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return final_summary[len(final_prompt):].strip(), combined_code_summary


for idx, row in code_summary_codellama_df.iterrows():
    bug_id = row['bug.id']
    combined_code_summary = str(row['code_summary']) if pd.notna(row['code_summary']) else ""
    bug_report = str(row['bug_report']) if pd.notna(row['bug_report']) else ""


    print(f"Processing Bug ID: {bug_id}")
    try:
        if combined_code_summary.strip():
            final_summary, combined_code_summary = generate_final_summary(combined_code_summary, bug_report)
            print(final_summary)
        else:
            final_summary = "No buggy code summary provided."
            combined_code_summary = ""
    except Exception as e:
        final_summary = f"Error: {str(e)}"
        combined_code_summary = ""


    summaries.append({
        "bug.id": bug_id,
        "summary": final_summary,
        "code_summary": combined_code_summary
    })
 
code_br_few_codellama_df = pd.DataFrame(summaries, columns=['bug.id', 'summary', 'code_summary'])

Processing Bug ID: 66
The issue is that the Closure Compiler is removing the parameters of a function, even if they are not used, when the compilation level is set to "Simple Optimizations".
Processing Bug ID: 67
Combining @interfaces with multiple extends can cause the compiler to crash when the extends are of unknown type.
Processing Bug ID: 68
The compiler is trying to inline a function that uses a variable that is only defined in its catch block, which is not allowed.
Processing Bug ID: 69
The code is trying to convert an object from one interface to another, and it is causing an infinite recursion.
Processing Bug ID: 70
The compiler is ignoring the delete statement when it is used to delete an object property that was previously assigned to a variable. This is causing the code to not function as intended.
Processing Bug ID: 71
The code declares two functions with the same name but different parameters, which is not allowed in JavaScript. This is a bug in the code, not the compiler

In [39]:
code_br_few_codellama_df.to_csv('code_br_few_codellama.csv', index=False)